In [5]:
import pandas as pd
import json
import sqlite3
from sqlalchemy import create_engine
import sqlalchemy.types

In [7]:
with open('yelp_academic_dataset_business.json', 'r', encoding='utf-8') as f:
    business_data = [json.loads(line) for line in f]
business_df = pd.DataFrame(business_data)

with open('yelp_academic_dataset_checkin.json', 'r', encoding='utf-8') as f:
    checkin_data = [json.loads(line) for line in f]
checkin_df = pd.DataFrame(checkin_data)

with open('yelp_academic_dataset_review.json', 'r', encoding='utf-8') as f:
    review_data = [json.loads(line) for line in f]
review_df = pd.DataFrame(review_data)

with open('yelp_academic_dataset_tip.json', 'r', encoding='utf-8') as f:
    tip_data = [json.loads(line) for line in f]
tip_df = pd.DataFrame(tip_data)

with open('yelp_academic_dataset_user.json', 'r', encoding='utf-8') as f:
    user_data = [json.loads(line) for line in f]
user_df = pd.DataFrame(user_data)

with open('photos.json', 'r', encoding='utf-8') as f:
    photos_data = [json.loads(line) for line in f]
photos_df = pd.DataFrame(photos_data)

In [8]:
print(business_df.shape)
print(checkin_df.shape)
print(review_df.shape)
print(tip_df.shape)
print(user_df.shape)
print(photos_df.shape)

(150346, 14)
(131930, 2)
(6990280, 9)
(908915, 5)
(1987897, 22)
(200100, 4)


In [31]:
# Create SQLAlchemy engine
engine = create_engine('sqlite:///yelp_entire.db')

# Convert dictionary columns to JSON strings
business_df['attributes'] = business_df['attributes'].apply(json.dumps)
business_df['hours'] = business_df['hours'].apply(json.dumps)

# Create a connection to SQLite database
conn = sqlite3.connect('yelp_entire.db')

# Write DataFrame to the SQLite database
business_df.to_sql('business', conn, if_exists='replace', index=True, 
                   dtype={'attributes': 'TEXT', 'hours': 'TEXT'}, chunksize=50000)

# Close the connection
conn.close()

In [39]:
def load_dataframe(df, table_name, engine):
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)

# Load each DataFrame into a separate table
load_dataframe(review_df, 'review', engine)
load_dataframe(user_df, 'user', engine)
load_dataframe(tip_df, 'tip', engine)
load_dataframe(checkin_df, 'checkin', engine)
load_dataframe(photos_df,'photo',engine)

In [41]:
import sqlite3
conn = sqlite3.connect('yelp_entire.db')

In [43]:
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'",conn)
tables

,name
0,business
1,review
2,user
3,tip
4,checkin
5,photo


In [45]:
# explore what type of data available in the tables
for table in tables['name']:
    print('-'*50,f'{table}','-'*50)
    display(pd.read_sql_query(f"select * from {table} limit 5",conn))

-------------------------------------------------- business --------------------------------------------------


,index,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,"{""ByAppointmentOnly"": ""True""}","Doctors, Traditional Chinese Medicine, Naturop...",null
1,1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,"{""BusinessAcceptsCreditCards"": ""True""}","Shipping Centers, Local Services, Notaries, Ma...","{""Monday"": ""0:0-0:0"", ""Tuesday"": ""8:0-18:30"", ..."
2,2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{""Monday"": ""8:0-22:0"", ""Tuesday"": ""8:0-22:0"", ..."
3,3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{""RestaurantsDelivery"": ""False"", ""OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{""Monday"": ""7:0-20:0"", ""Tuesday"": ""7:0-20:0"", ..."
4,4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{""BusinessAcceptsCreditCards"": ""True"", ""Wheelc...","Brewpubs, Breweries, Food","{""Wednesday"": ""14:0-22:0"", ""Thursday"": ""16:0-2..."


-------------------------------------------------- review --------------------------------------------------


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


-------------------------------------------------- user --------------------------------------------------


...

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


-------------------------------------------------- checkin --------------------------------------------------


,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


-------------------------------------------------- photo --------------------------------------------------


,photo_id,business_id,caption,label
0,zsvj7vloL4L5jhYyPIuVwg,Nk-SJhPlDBkAZvfsADtccA,Nice rock artwork everywhere and craploads of ...,inside
1,HCUdRJHHm_e0OCTlZetGLg,yVZtL5MmrpiivyCIrVkGgA,,outside
2,vkr8T0scuJmGVvN2HJelEA,_ab50qdWOk0DdB6XOrBitw,oyster shooter,drink
3,pve7D6NUrafHW3EAORubyw,SZU9c8V2GuREDN5KgyHFJw,Shrimp scampi,food
4,H52Er-uBg6rNrHcReWTD2w,Gzur0f0XMkrVxIwYJvOt2g,,food


In [47]:
# total business count
pd.read_sql_query("select count(*) from business ",conn)

,count(*)
0,150346


In [49]:
# total business count
pd.read_sql_query("select count(*) from review ",conn)

,count(*)
0,6990280


In [51]:
# total business count
pd.read_sql_query("select count(*) from user ",conn)

,count(*)
0,1987897


In [53]:
# total business count
pd.read_sql_query("select count(*) from tip ",conn)

,count(*)
0,908915


In [55]:
# total business count
pd.read_sql_query("select count(*) from checkin ",conn)

,count(*)
0,131930


In [57]:
# total business count
pd.read_sql_query("select count(*) from photo ",conn)

,count(*)
0,200100


In [114]:
def split_and_explode(row):
    categories = row['categories']
    if categories is not None and categories.strip() != '':
        categories = categories.split(', ')
        return pd.Series({'business_id': row['business_id'], 'business_category': categories})
    else:
        return pd.Series({'business_id': row['business_id'], 'business_category': []})

# Apply the function to split and explode categories
business_categories = business_df.apply(split_and_explode, axis=1)

# Explode the list of business categories
business_categories = business_categories.explode('business_category')

# Display the resulting DataFrame
print(business_categories)

                   business_id             business_category
0       Pns2l4eNsfO8kk83dixA6A                       Doctors
0       Pns2l4eNsfO8kk83dixA6A  Traditional Chinese Medicine
0       Pns2l4eNsfO8kk83dixA6A         Naturopathic/Holistic
0       Pns2l4eNsfO8kk83dixA6A                   Acupuncture
0       Pns2l4eNsfO8kk83dixA6A              Health & Medical
...                        ...                           ...
150344  mtGm22y5c2UHNXDFAjaPNw                         Bikes
150345  jV_XOycEzSlTx-65W906pg                 Beauty & Spas
150345  jV_XOycEzSlTx-65W906pg              Permanent Makeup
150345  jV_XOycEzSlTx-65W906pg                      Piercing
150345  jV_XOycEzSlTx-65W906pg                        Tattoo

[668695 rows x 2 columns]


In [116]:
def extract_and_explode(row):
    attributes = row['attributes']
    if isinstance(attributes, dict):
        attribute_list = [key for key, value in attributes.items() if value]
        return pd.Series({'business_id': row['business_id'], 'business_attribute': attribute_list})
    else:
        return pd.Series({'business_id': row['business_id'], 'business_attribute': []})

# Apply the function to extract and explode attributes
business_attributes = business_df.apply(extract_and_explode, axis=1)

# Explode the list of business attributes
business_attributes = business_attributes.explode('business_attribute')

# Display the resulting DataFrame
print(business_attributes)

KeyError: 'attributes'

In [ ]:
rows = []

# Define the days of the week
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Iterate over rows in the DataFrame
for _, row in business_df.iterrows():
    business_id = row['business_id']
    hours = row['hours']
    
    if hours is not None:
        for day_of_week in days_of_week:
            time_range = hours.get(day_of_week)
            if time_range:
                open_time, close_time = time_range.split('-')
                rows.append({
                    'business_id': business_id,
                    'day_of_week': day_of_week,
                    'open_time': open_time,
                    'close_time': close_time
                })

# Create a DataFrame from the list of rows
business_hours = pd.DataFrame(rows)

# Display the resulting DataFrame
print(business_hours)

In [118]:
import os

# Path to the SQLite database file
db_file_path = 'yelp.db'

# Check if the file exists
if os.path.exists(db_file_path):
    # Delete the SQLite database file
    os.remove(db_file_path)
    print(f"Database '{db_file_path}' has been deleted.")
else:
    print(f"Database '{db_file_path}' does not exist.")


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'yelp.db'